<a href="https://colab.research.google.com/github/rrfsantos/Large-Language-Model-Agents-MOOC-Fall-2024/blob/main/AI%20Agentic%20Design%20Patterns%20with%20AutoGen/L3_Reflection_and_Blogpost_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lesson 3: Reflection and Blogpost Writing

## Setup

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#!pip install -r /content/drive/MyDrive/AutoGen/requirements.txt

In [27]:
from dotenv import load_dotenv
import warnings
import os
from autogen import ConversableAgent, initiate_chats
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [28]:
# Load environment variables from .env file
load_dotenv("/content/drive/MyDrive/AutoGen/env")
llm_config = {"config_list": [{"model": "gpt-3.5-turbo", "api_key" : os.getenv("OPENAI_API_KEY")}]}

## The task!

In [29]:
task = '''
        Write a concise but engaging blogpost about
       LLM Agents. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [30]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise "
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

[autogen.oai.client: 10-28 16:15:39] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [31]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [32]:
print(reply)

Title: Unveiling the Power of LLM Agents

Delving into the world of artificial intelligence, LLM (Large Language Models) agents are taking the tech industry by storm. These advanced AI models are trained to understand and generate human language, revolutionizing how we interact with technology. From smart assistants to content generation, LLM agents are at the forefront of innovation. Their ability to process enormous amounts of data enables them to provide efficient and accurate responses. As we continue to witness their potential unfold, LLM agents are reshaping the way we communicate and engage with the digital world. Exciting times lie ahead as we embrace the era of LLM agents.


## Adding reflection

Create a critic agent to reflect on the work of the writer agent.

In [33]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

[autogen.oai.client: 10-28 16:15:41] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [34]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       LLM Agents. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of LLM Agents

Delving into the world of artificial intelligence, LLM (Large Language Models) agents are taking the tech industry by storm. These advanced AI models are trained to understand and generate human language, revolutionizing how we interact with technology. From smart assistants to content generation, LLM agents are at the forefront of innovation. Their ability to process enormous amounts of data enables them to provide efficient and accurate responses. As we continue to witness their potential unfold, LLM agents are reshaping the way we communicate and engage with the digital world. Exciting times lie ahead as we embrace the era of LLM agents.

-----------------------------------------------------

## Nested chat

In [35]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


[autogen.oai.client: 10-28 16:15:45] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [36]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

[autogen.oai.client: 10-28 16:15:45] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [37]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

[autogen.oai.client: 10-28 16:15:45] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [38]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

[autogen.oai.client: 10-28 16:15:45] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Orchestrate the nested chats to solve the task

In [39]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content.
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer,
     "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message,
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" :
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer,
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.",
     "max_turns": 1},
]


In [40]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [41]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       LLM Agents. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of LLM Agents

Delving into the world of artificial intelligence, LLM (Large Language Models) agents are taking the tech industry by storm. These advanced AI models are trained to understand and generate human language, revolutionizing how we interact with technology. From smart assistants to content generation, LLM agents are at the forefront of innovation. Their ability to process enormous amounts of data enables them to provide efficient and accurate responses. As we continue to witness their potential unfold, LLM agents are reshaping the way we communicate and engage with the digital world. Exciting times lie ahead as we embrace the era of LLM agents.

-----------------------------------------------------

## Get the summary

In [42]:
print(res.summary)

Title: Navigating the Impact of LLM Agents: Balancing Innovation and Responsibility

Embark on a journey through the frontier of artificial intelligence with LLM (Large Language Models) agents. These cutting-edge AI systems are reshaping the digital landscape, from enhancing user experiences to enabling efficient information processing. As we marvel at their capabilities, it's crucial to understand the ethical implications that accompany this technological advancement. Delve into discussions on data privacy, bias mitigation, and societal consequences to navigate the responsible use of LLM agents. Join the conversation on embracing innovation while upholding ethical standards in the era of transformative AI technologies. Let's embark on this thoughtful exploration together.
